In [75]:
import numpy as np 
import pandas as pd 
import spacy 
from textblob import TextBlob

df = pd.read_csv('Amazon_Product_Reviews.csv')
df.head()

,id,dateAdded,dateUpdated,name,asins,brand,categories,primaryCategories,imageURLs,keys,...,reviews.dateSeen,reviews.doRecommend,reviews.id,reviews.numHelpful,reviews.rating,reviews.sourceURLs,reviews.text,reviews.title,reviews.username,sourceURLs
0,AVqVGZNvQMlgsOJE6eUY,2017-03-03T16:56:05Z,2018-10-25T16:36:31Z,"Amazon Kindle E-Reader 6"" Wifi (8th Generation...",B00ZV9PXP2,Amazon,"Computers,Electronics Features,Tablets,Electro...",Electronics,https://pisces.bbystatic.com/image2/BestBuy_US...,allnewkindleereaderblack6glarefreetouchscreend...,...,"2018-05-27T00:00:00Z,2017-09-18T00:00:00Z,2017...",False,NaN,0,3,http://reviews.bestbuy.com/3545/5442403/review...,I thought it would be as big as small paper bu...,Too small,llyyue,https://www.newegg.com/Product/Product.aspx%25...
1,AVqVGZNvQMlgsOJE6eUY,2017-03-03T16:56:05Z,2018-10-25T16:36:31Z,"Amazon Kindle E-Reader 6"" Wifi (8th Generation...",B00ZV9PXP2,Amazon,"Computers,Electronics Features,Tablets,Electro...",Electronics,https://pisces.bbystatic.com/image2/BestBuy_US...,allnewkindleereaderblack6glarefreetouchscreend...,...,"2018-05-27T00:00:00Z,2017-07-07T00:00:00Z,2017...",True,NaN,0,5,http://reviews.bestbuy.com/3545/5442403/review...,This kindle is light and easy to use especiall...,Great light reader. Easy to use at the beach,Charmi,https://www.newegg.com/Product/Product.aspx%25...
2,AVqVGZNvQMlgsOJE6eUY,2017-03-03T16:56:05Z,2018-10-25T16:36:31Z,"Amazon Kindle E-Reader 6"" Wifi (8th Generation...",B00ZV9PXP2,Amazon,"Computers,Electronics Features,Tablets,Electro...",Electronics,https://pisces.bbystatic.com/image2/BestBuy_US...,allnewkindleereaderblack6glarefreetouchscreend...,...,2018-05-27T00:00:00Z,True,NaN,0,4,https://reviews.bestbuy.com/3545/5442403/revie...,Didnt know how much i'd use a kindle so went f...,Great for the price,johnnyjojojo,https://www.newegg.com/Product/Product.aspx%25...
3,AVqVGZNvQMlgsOJE6eUY,2017-03-03T16:56:05Z,2018-10-25T16:36:31Z,"Amazon Kindle E-Reader 6"" Wifi (8th Generation...",B00ZV9PXP2,Amazon,"Computers,Electronics Features,Tablets,Electro...",Electronics,https://pisces.bbystatic.com/image2/BestBuy_US...,allnewkindleereaderblack6glarefreetouchscreend...,...,2018-10-09T00:00:00Z,True,177283626.0,3,5,https://redsky.target.com/groot-domain-api/v1/...,I am 100 happy with my purchase. I caught it o...,A Great Buy,Kdperry,https://www.newegg.com/Product/Product.aspx%25...
4,AVqVGZNvQMlgsOJE6eUY,2017-03-03T16:56:05Z,2018-10-25T16:36:31Z,"Amazon Kindle E-Reader 6"" Wifi (8th Generation...",B00ZV9PXP2,Amazon,"Computers,Electronics Features,Tablets,Electro...",Electronics,https://pisces.bbystatic.com/image2/BestBuy_US...,allnewkindleereaderblack6glarefreetouchscreend...,...,2018-05-27T00:00:00Z,True,NaN,0,5,https://reviews.bestbuy.com/3545/5442403/revie...,Solid entry level Kindle. Great for kids. Gift...,Solid entry-level Kindle. Great for kids,Johnnyblack,https://www.newegg.com/Product/Product.aspx%25...


In [76]:
df.columns

Index(['id', 'dateAdded', 'dateUpdated', 'name', 'asins', 'brand',
       'categories', 'primaryCategories', 'imageURLs', 'keys', 'manufacturer',
       'manufacturerNumber', 'reviews.date', 'reviews.dateAdded',
       'reviews.dateSeen', 'reviews.doRecommend', 'reviews.id',
       'reviews.numHelpful', 'reviews.rating', 'reviews.sourceURLs',
       'reviews.text', 'reviews.title', 'reviews.username', 'sourceURLs'],
      dtype='object')

In [77]:
# Select only needed columns and drop all unnecessary columns
cleaned = df[['reviews.text']]
cleaned.head()

,reviews.text
0,I thought it would be as big as small paper bu...
1,This kindle is light and easy to use especiall...
2,Didnt know how much i'd use a kindle so went f...
3,I am 100 happy with my purchase. I caught it o...
4,Solid entry level Kindle. Great for kids. Gift...


In [78]:
# Check for null values in cleaned
cleaned.isnull().sum()

reviews.text    0
dtype: int64

In [79]:
# Obtain just the reviews
text = cleaned['reviews.text']
text

0       I thought it would be as big as small paper bu...
1       This kindle is light and easy to use especiall...
2       Didnt know how much i'd use a kindle so went f...
3       I am 100 happy with my purchase. I caught it o...
4       Solid entry level Kindle. Great for kids. Gift...
                              ...                        
4995    This is a great tablet for the price. Amazon i...
4996    This tablet is the perfect size and so easy to...
4997    Purchased this for my son. Has room to upgrade...
4998    I had some thoughts about getting this for a 5...
4999    this is a steal, have 8 gb model as well.This ...
Name: reviews.text, Length: 5000, dtype: object

In [80]:
# Load Language Package
nlp = spacy.load('en_core_web_sm')

In [81]:
# creating a function to preprocess text
def preprocess(text):
    doc = nlp(text.lower().strip())
    processed = [token.lemma_.lower() for token in doc if not token.is_stop and not token.is_punct]
        
    return ' '.join(processed)

In [82]:
cleaned['processed.text'] = cleaned['reviews.text'].apply(preprocess)

C:\Users\DELL\AppData\Local\Temp\ipykernel_9072\1707223750.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleaned['processed.text'] = cleaned['reviews.text'].apply(preprocess)


In [83]:
cleaned.head()

,reviews.text,processed.text
0,I thought it would be as big as small paper bu...,think big small paper turn like palm think sma...
1,This kindle is light and easy to use especiall...,kindle light easy use especially beach
2,Didnt know how much i'd use a kindle so went f...,not know use kindle go low end m happy little ...
3,I am 100 happy with my purchase. I caught it o...,100 happy purchase catch sale good price norma...
4,Solid entry level Kindle. Great for kids. Gift...,solid entry level kindle great kid gift kid fr...


In [84]:
def analyze_polarity(text):
    # preprocess the text with spacy
    doc = nlp(text)
    
    # Analyze sentiment with TextBlob
    blob = TextBlob(text)
    polarity = blob.sentiment.polarity
    
    return polarity


In [85]:
def polarity(text):
    doc = nlp(text)
    polarity_value = doc._.blob.polarity
    sentiment_value =doc._.blob.sentiment
    
    return polarity_value, sentiment_value


In [86]:
data = cleaned['processed.text'].values
data

array(['think big small paper turn like palm think small read comfortable regular kindle definitely recommend paperwhite instead',
       'kindle light easy use especially beach',
       'not know use kindle go low end m happy little dark', ...,
       'purchase son room upgrade memory allow book game speaker well locate well position',
       'thought get 5 year old screen protector case feel long time',
       'steal 8 gb model well.this punch'], dtype=object)

In [87]:
data = cleaned['processed.text'].values
sentiments = []

for item in data:
    polarity_score = analyze_polarity(item)
    
    if polarity_score > 0:
        sentiment = 'positive'
    elif polarity_score < 0:
        sentiment = 'negative'
    else:
        sentiment = 'neutral'
    
    sentiments.append(sentiment)

In [88]:
sentiments

['negative',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'negative',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'neutral',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'negative',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'neutral',
 'positive',
 'positive',
 'positive',
 'neutral',
 'positive',
 'positive',
 'positive',
 'positive',
 'negative',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 '

In [90]:
positive_count = sentiments.count('positive')
negative_count = sentiments.count('negative')
neutral_count = sentiments.count('neutral')

total = len(sentiments)

positive_percentage = (positive_count / total)*100
negative_percentage = (negative_count / total)*100
neutral_percentage = (neutral_count / total)*100

print(positive_count)
print(negative_count)
print(neutral_count)
print(total)

print(f"Positive percentage: {positive_percentage:.2f}%")
print(f"Negative percentage: {negative_percentage:.2f}%")
print(f"Neutral percentage: {neutral_percentage:.2f}%")



4441
250
309
5000
Positive percentage: 88.82%
Negative percentage: 5.00%
Neutral percentage: 6.18%


In [96]:
# Test sentiment analysis function on sample reviews
sample_reviews = [
    "This product is amazing and works perfectly!",
    "I'm very disappointed with this item, it doesn't work as expected.",
    "The product is neither good nor bad.",
    "I don't know how i feel about the product."]

text_ = sample_reviews

# creating a function to preprocess text_
def preprocess(text_):
    doc = nlp(text_.lower().strip())
    processed = [token.lemma_.lower() for token in doc if not token.is_stop and not token.is_punct]
        
    return ' '.join(processed)



def analyze_polarity(text_):
    # preprocess the text_ with spacy
    doc = nlp(text_)
    
    # Analyze sentiment with TextBlob
    blob = TextBlob(text_)
    polarity = blob.sentiment.polarity
    
    return polarity

def polarity(text_):
    doc = nlp(text_)
    polarity_value = doc._.blob.polarity
    sentiment_value =doc._.blob.sentiment
    
    return polarity_value, sentiment_value


sentiments = []

for item in text_:
    polarity_score = analyze_polarity(item)
    
    if polarity_score > 0:
        sentiment = 'positive'
    elif polarity_score < 0:
        sentiment = 'negative'
    else:
        sentiment = 'neutral'
    
    sentiments.append(sentiment)


In [97]:
sentiments

['positive', 'negative', 'positive', 'neutral']

In [98]:
positive_count = sentiments.count('positive')
negative_count = sentiments.count('negative')
neutral_count = sentiments.count('neutral')

total = len(sentiments)

positive_percentage = (positive_count / total)*100
negative_percentage = (negative_count / total)*100
neutral_percentage = (neutral_count / total)*100

print(positive_count)
print(negative_count)
print(neutral_count)
print(total)

print(f"Positive percentage: {positive_percentage:.2f}%")
print(f"Negative percentage: {negative_percentage:.2f}%")
print(f"Neutral percentage: {neutral_percentage:.2f}%")

2
1
1
4
Positive percentage: 50.00%
Negative percentage: 25.00%
Neutral percentage: 25.00%
